# 🩺📊✨ Text Analytics for Health

Azure AI Language Text Analytics for Health is a cloud-based API service that uses machine learning to extract and label relevant medical information from unstructured texts like doctor's notes, discharge summaries, clinical documents, and electronic health records. This service is designed to help healthcare providers improve health outcomes by analyzing text data for insights.

Its documentation can be found [here](https://learn.microsoft.com/en-us/azure/ai-services/language-service/text-analytics-for-health/overview?tabs=ner). 

With Azure AI Language Text Analytics for Health, you can:

- **Named Entity Recognition**: Identify and categorize medical entities such as 
    - *Medications*: Names of drugs, dosages, and forms (e.g., tablets, injections).
    - *Conditions*: Medical conditions, diseases, symptoms, and diagnoses.
    - *Procedures*: Medical procedures, surgeries, and treatments.
    - *Anatomical Terms*: Parts of the body, organs, and tissues.
    - *Lab Tests*: Names of laboratory tests and their results.
    - *Medical Devices*: Equipment and devices used in medical treatments.
    - *Healthcare Providers*: Names and roles of healthcare professionals (e.g., doctors, nurses).
    - *Patient Information*: Demographic details such as age, gender, and ethnicity.
    - *Clinical Events*: Events related to patient care, such as hospital admissions and discharges.
- **Relation Extraction**: Determine relationships between entities, such as dosage and medication.
- **Entity Linking**: Link entities to standardized medical vocabularies like UMLS.
- **Assertion Detection**: Identify whether entities are present, absent, or conditional.
- **Social Determinants of Health (SDOH) Extraction**: Extract mentions of social factors affecting health, such as living conditions and ethnicity

## How to run this notebook

1. Make sure Python is installed. 
2. Create a virtual environment and activate it. 
3. Install the dependencies specified in requirements.txt.

In [1]:
!pip install -r requirements.txt

4. Deploy the necessary services by clicking on the Deploy button in the notebook.


<a href="https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2FNicoGrassetto%2Fai-language-notebooks%2Fmain%2Ftext-analytics-for-health%2Fdeplo.json" target="_blank">
    <img src="deploytoazure.svg" alt="Button" style="width:200px;height:auto;">
</a>

In [3]:
from azure.identity import DefaultAzureCredential
from azure.ai.textanalytics import TextAnalyticsClient
import os
# Use DefaultAzureCredential for managed identity authentication
credential = DefaultAzureCredential()
endpoint = os.environ.get('LANGUAGE_ENDPOINT')
type(endpoint)
# # Create the Text Analytics client
# client = TextAnalyticsClient(endpoint=endpoint, credential=credential)

NoneType

In [ ]:
# # Authenticate the client using your key and endpoint 
# def authenticate_client():
#     ta_credential = AzureKeyCredential(key)
#     text_analytics_client = TextAnalyticsClient(
#             endpoint=endpoint, 
#             credential=ta_credential)
#     return text_analytics_client

# client = authenticate_client()

# # Example function for extracting information from healthcare-related text 
# def health_example(client):
#     documents = [
#         """
#         Patient needs to take 50 mg of ibuprofen.
#         """
#     ]

#     poller = client.begin_analyze_healthcare_entities(documents)
#     result = poller.result()

#     docs = [doc for doc in result if not doc.is_error]

#     for idx, doc in enumerate(docs):
#         for entity in doc.entities:
#             print("Entity: {}".format(entity.text))
#             print("...Normalized Text: {}".format(entity.normalized_text))
#             print("...Category: {}".format(entity.category))
#             print("...Subcategory: {}".format(entity.subcategory))
#             print("...Offset: {}".format(entity.offset))
#             print("...Confidence score: {}".format(entity.confidence_score))
#         for relation in doc.entity_relations:
#             print("Relation of type: {} has the following roles".format(relation.relation_type))
#             for role in relation.roles:
#                 print("...Role '{}' with entity '{}'".format(role.name, role.entity.text))
#         print("------------------------------------------")
# health_example(client)

: 